# Migrate SQL Server Database to Azure SQL DB

## Overview

Use this notebook when attempting to migrate a SQL Server database to Azure SQL Database. The process is different than using an Azure SQL Virtual Machine or Azure SQL Managed Instance. Refer to this [Microsoft docs](https://docs.microsoft.com/en-us/azure/dms/howto-sql-server-to-azure-sql-powershell) article for the method described. 

## Variables
Use the table below as reference when configuring this notebook for execution. 

| Line | Variable | Description | Example 
| --- | --- | --- | --- 
|  | Subscription | Azure subscription name or ID | guid 
|  | ResourceGroup | Exact name of resource group to target or create | *MyAz_RG* 
|  | Location |  |  |
|  | StorageName |  |  |
|  | StorageSKU |  |  |
|  | ServiceName | Service name for the DMS instance | *MyDMS* 
|  | ProjectName | Name of Azure Database Migration Service project in which to create the task.
|  | TaskType | Type of migration task to create for SQL Server to Azure SQL Database migration type MigrateSqlServerSqlDb is expected.
|  | TaskName | Name of task to be created. | *MigrationTask*

In [ ]:
$Subscription = ""
$ResourceGroup = ""
$Location = ""
$StorageName = ""
$StorageSKU = ""
$DmsServiceName = ""
$ProjectName = ""
$TaskType = ""
$TaskName = ""

## Steps

1. Use the Assessment Notebooks to scan the SQL Instance to be migrated and to prepare for database migration
2. Fix issues based on the assessments and continue to scan for issues until a clean assessment is made. This may take several hours, days, or even months depending on the size and complexity of the data; so plan accordingly.
3. Create a local backup directory according to the [Create SQL Backups Notebook](.\create-sql-backups.ipynb)
4. Specify the backup directory and other variables for this notebook
5. Run the cells below to migrate bacpac backup files in the backup folder to SQL Azure Database